# Setup

## Import libraries & settings

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import sys, os

In [2]:
sys.path.append("../")

In [3]:
from scripts.data_scraper import get_all_financial_data

Load the company metadata

In [4]:
raw_companies = pd.read_csv("../data/raw/companies.csv")

In [5]:
raw_companies.head()

,company_name,region,market_cap,ticker,sector
0,Exxon Mobil Corporation,United States,Large-Cap,XOM,Energy
1,Royal Dutch Shell plc,UK/Netherlands,Large-Cap,RDS.A,Energy
2,PetroChina Company Limited,China,Large-Cap,PTR,Energy
3,BP p.l.c.,United Kingdom,Large-Cap,BP,Energy
4,TotalEnergies SE,France,Large-Cap,TTE,Energy


In [6]:
# check missing data
raw_companies.isna().sum()

company_name    0
region          0
market_cap      0
ticker          0
sector          0
dtype: int64

In [7]:
print(f"Shape: {raw_companies.shape}\nUnique Tickers: {raw_companies['ticker'].nunique()}")

Shape: (455, 5)
Unique Tickers: 451


In [8]:
raw_companies['ticker'].duplicated().sum()

4

In [9]:
# removing duplicate tickers
raw_companies = raw_companies.drop_duplicates(subset='ticker')
print(f"Shape: {raw_companies.shape}\nUnique Tickers: {raw_companies['ticker'].nunique()}")

Shape: (451, 5)
Unique Tickers: 451


# Financial Data

Scrape the financial data for each company

In [10]:
fin_df = get_all_financial_data(raw_companies['ticker'].tolist())

In [11]:
fin_df.head()

,index,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,Net Interest Income,...,Fixed Assets Revaluation Reserve,Net Policyholder Benefits And Claims,Purchase Of Investment Properties,Net Income From Tax Loss Carryforward,Current Deferred Taxes Assets,Excess Tax Benefit From Stock Based Compensation,Change In Interest Payable,Receiptsfrom Government Grants,Securities Amortization,Dividends Paid Direct
0,2023-12-31,0.0,0.33,74273000000.0,36010000000.0,20641000000.0,250555000000.0,74273000000.0,53632000000.0,-849000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-31,0.0,0.33,102591000000.0,55740000000.0,24040000000.0,295608000000.0,102591000000.0,78551000000.0,-798000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-31,0.0,0.31,52788000000.0,23040000000.0,20607000000.0,211806000000.0,52788000000.0,32181000000.0,-947000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-31,0.0,0.17,18284000000.0,-22440000000.0,46009000000.0,170447000000.0,18284000000.0,-27725000000.0,-1158000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-12-31,-2073390000.0,0.33,49786000000.0,15239000000.0,15928000000.0,162058000000.0,43503000000.0,27575000000.0,-1915000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
fin_df['Ticker'].nunique()

402

In [13]:
# compare the available companies after scraping financial data
lost_companies = list(set(raw_companies['ticker'].unique()).difference(set(fin_df['Ticker'].unique())))

In [14]:
raw_companies.loc[raw_companies['ticker'].isin(lost_companies)]

,company_name,region,market_cap,ticker,sector
1,Royal Dutch Shell plc,UK/Netherlands,Large-Cap,RDS.A,Energy
2,PetroChina Company Limited,China,Large-Cap,PTR,Energy
10,Gazprom PJSC,Russia,Large-Cap,OGZPY,Energy
12,Woodside Petroleum Ltd,Australia,Large-Cap,WPL.AX,Energy
15,Marathon Oil Corporation,United States,Mid-Cap,MRO,Energy
17,Oil Search Limited,Papua New Guinea,Mid-Cap,OSH.AX,Energy
22,CNOOC Limited,China,Mid-Cap,CEO,Energy
23,Husky Energy Inc.,Canada,Mid-Cap,HSE.TO,Energy
32,African Petroleum Corporation Ltd,Australia,Small-Cap,APCL,Energy
34,Wentworth Resources plc,Tanzania,Small-Cap,WEN.L,Energy


# ESG Data

Scrape the ESG Scores for each company

In [15]:
raw_companies.sample()

,company_name,region,market_cap,ticker,sector
416,Telkom SA SOC Ltd.,South Africa,Small-Cap,TKG.JO,Communication Services


In [16]:
esg_ticker = yf.Ticker("SU.PA")

In [17]:
sus = esg_ticker.sustainability.T

In [18]:
# add the ticker column
sus['Ticker'] = "SU.PA"

In [19]:
sus

,maxAge,totalEsg,environmentScore,socialScore,governanceScore,ratingYear,ratingMonth,highestControversy,peerCount,esgPerformance,...,furLeather,gambling,gmo,militaryContract,nuclear,pesticides,palmOil,coal,tobacco,Ticker
esgScores,86400,10.04,3.6,3.95,2.5,2025,2,2.0,109,LAG_PERF,...,False,False,False,False,False,False,False,False,False,SU.PA


In [20]:
# add the rating date using the ratingYear and ratingMonth
sus['ratingDate'] = pd.to_datetime(sus['ratingYear'].astype(str) + '-' + sus['ratingMonth'].astype(str), format='%Y-%m')

In [21]:
sus

,maxAge,totalEsg,environmentScore,socialScore,governanceScore,ratingYear,ratingMonth,highestControversy,peerCount,esgPerformance,...,gambling,gmo,militaryContract,nuclear,pesticides,palmOil,coal,tobacco,Ticker,ratingDate
esgScores,86400,10.04,3.6,3.95,2.5,2025,2,2.0,109,LAG_PERF,...,False,False,False,False,False,False,False,False,SU.PA,2025-02-01


In [22]:
def scrape_esg(tickers):
    company_data = []  # list to store each company's data as their own df
    
    for ticker_str in tickers:
        ticker = yf.Ticker(ticker_str)

        df_temp = ticker.sustainability.T
        df_temp['Ticker'] = ticker_str

        company_data.append(df_temp)

    df_final = pd.concat(company_data)
    return df_final

In [23]:
esg_df = scrape_esg(raw_companies['ticker'].tolist())

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RDS.A?modules=esgScores&corsDomain=finance.yahoo.com&formatted=false&symbol=RDS.A&crumb=N07q.Qb24kq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PTR?modules=esgScores&corsDomain=finance.yahoo.com&formatted=false&symbol=PTR&crumb=N07q.Qb24kq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BP?modules=esgScores&corsDomain=finance.yahoo.com&formatted=false&symbol=BP&crumb=N07q.Qb24kq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TTE?modules=esgScores&corsDomain=finance.yahoo.com&formatted=false&symbol=TTE&crumb=N07q.Qb24kq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/2222.SR?modules=esgScores&corsDomain=finance.yahoo.com&formatted=false&symbol=2222.SR&crumb=N07q.Qb24kq
404 Client Error: Not Found for url

In [24]:
esg_df

,maxAge,totalEsg,environmentScore,socialScore,governanceScore,ratingYear,ratingMonth,highestControversy,peerCount,esgPerformance,...,furLeather,gambling,gmo,militaryContract,nuclear,pesticides,palmOil,coal,tobacco,Ticker
esgScores,86400,43.66,25.09,11.58,6.99,2025,2,3.0,165,LEAD_PERF,...,False,False,False,False,False,False,False,False,False,XOM
esgScores,86400,38.36,19.91,10.61,7.84,2025,2,3.0,165,LEAD_PERF,...,False,False,False,False,False,False,False,False,False,CVX
esgScores,86400,41.04,24.19,10.65,6.2,2023,9,2.0,4,LEAD_PERF,...,False,False,False,False,False,False,False,False,False,RELIANCE.NS
esgScores,86400,32.57,17.96,7.32,7.29,2025,2,2.0,165,LEAD_PERF,...,False,False,False,False,False,False,False,False,False,CNQ
esgScores,86400,36.81,23.92,8.48,4.41,2025,2,3.0,165,LEAD_PERF,...,False,False,False,False,False,False,False,False,False,SU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
esgScores,86400,8.15,2.44,2.91,2.8,2025,2,1.0,512,LAG_PERF,...,False,False,False,False,False,False,False,False,False,C09.SI
esgScores,86400,13.82,2.3,4.33,7.19,2025,2,1.0,512,LAG_PERF,...,False,False,False,False,False,False,False,False,False,VNO
esgScores,86400,13.66,2.67,4.57,6.42,2025,2,0.0,512,LAG_PERF,...,False,False,False,False,False,False,False,False,False,0688.HK
esgScores,86400,20.23,6.31,8.27,5.65,2025,2,0.0,512,AVG_PERF,...,False,False,False,False,False,False,False,False,False,U14.SI


In [25]:
esg_df.shape

(360, 37)

In [26]:
lost_companies_2 = list(set(raw_companies['ticker'].unique()).difference(set(esg_df['Ticker'].unique())))

In [27]:
raw_companies.loc[raw_companies['ticker'].isin(lost_companies_2)]

,company_name,region,market_cap,ticker,sector
1,Royal Dutch Shell plc,UK/Netherlands,Large-Cap,RDS.A,Energy
2,PetroChina Company Limited,China,Large-Cap,PTR,Energy
3,BP p.l.c.,United Kingdom,Large-Cap,BP,Energy
4,TotalEnergies SE,France,Large-Cap,TTE,Energy
6,Saudi Arabian Oil Company (Aramco),Saudi Arabia,Large-Cap,2222.SR,Energy
...,...,...,...,...,...
428,Unibail-Rodamco-Westfield SE,France,Large-Cap,URW.AS,Real Estate
435,CapitaLand Limited,Singapore,Mid-Cap,C31.SI,Real Estate
445,Growthpoint Properties Limited,South Africa,Small-Cap,GRTJ.J,Real Estate
449,"Ayala Land, Inc.",Philippines,Small-Cap,ALI.PS,Real Estate


In [28]:
raw_companies.loc[~raw_companies['ticker'].isin(lost_companies_2), 'market_cap'].value_counts()

market_cap
Large-Cap    132
Mid-Cap      126
Small-Cap    102
Name: count, dtype: int64

Merging financials w/ esg

In [29]:
fin_df.rename(columns={"index": "Date"}, inplace=True)

In [30]:
foo = fin_df.merge(esg_df, how='left', on='Ticker')

In [31]:
foo.insert(0, "Ticker", foo.pop("Ticker"))

C:\Users\Mishaal\AppData\Local\Temp\ipykernel_8780\1978444490.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  foo.insert(0, "Ticker", foo.pop("Ticker"))


In [32]:
foo.insert(2, "totalEsg", foo.pop("totalEsg"))

C:\Users\Mishaal\AppData\Local\Temp\ipykernel_8780\1354071956.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  foo.insert(2, "totalEsg", foo.pop("totalEsg"))


In [33]:
# drop rows with missing esgScore
foo = foo.dropna(subset="totalEsg")
foo.shape

(1530, 377)

In [34]:
foo['ratingDate'] = pd.to_datetime(foo['ratingYear'].astype(str) + '-' + foo['ratingMonth'].astype(str), format='%Y-%m')

C:\Users\Mishaal\AppData\Local\Temp\ipykernel_8780\616063362.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  foo['ratingDate'] = pd.to_datetime(foo['ratingYear'].astype(str) + '-' + foo['ratingMonth'].astype(str), format='%Y-%m')


In [35]:
foo

,Ticker,Date,totalEsg,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,...,furLeather,gambling,gmo,militaryContract,nuclear,pesticides,palmOil,coal,tobacco,ratingDate
0,XOM,2023-12-31,43.66,0.0,0.33,74273000000.0,36010000000.0,20641000000.0,250555000000.0,74273000000.0,...,False,False,False,False,False,False,False,False,False,2025-02-01
1,XOM,2022-12-31,43.66,0.0,0.33,102591000000.0,55740000000.0,24040000000.0,295608000000.0,102591000000.0,...,False,False,False,False,False,False,False,False,False,2025-02-01
2,XOM,2021-12-31,43.66,0.0,0.31,52788000000.0,23040000000.0,20607000000.0,211806000000.0,52788000000.0,...,False,False,False,False,False,False,False,False,False,2025-02-01
3,XOM,2020-12-31,43.66,0.0,0.17,18284000000.0,-22440000000.0,46009000000.0,170447000000.0,18284000000.0,...,False,False,False,False,False,False,False,False,False,2025-02-01
12,CVX,2023-12-31,38.36,0.0,0.276,45042000000.0,21369000000.0,14989000000.0,136522000000.0,45042000000.0,...,False,False,False,False,False,False,False,False,False,2025-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,U14.SI,2020-12-31,20.23,-13214490.295817,0.214936,359302000.0,13141000.0,117880000.0,1249563000.0,297821000.0,...,False,False,False,False,False,False,False,False,False,2025-02-01
1700,8804.T,2023-12-31,16.92,-349140000.0,0.33,97417000000.0,45084000000.0,20686000000.0,266829000000.0,96359000000.0,...,False,False,False,False,False,False,False,False,False,2025-02-01
1701,8804.T,2022-12-31,16.92,-232023585.657371,0.297466,88737000000.0,43062000000.0,19113000000.0,248452000000.0,87957000000.0,...,False,False,False,False,False,False,False,False,False,2025-02-01
1702,8804.T,2021-12-31,16.92,3595673169.257033,0.360903,71800000000.0,34965000000.0,18862000000.0,247933000000.0,81763000000.0,...,False,False,False,False,False,False,False,False,False,2025-02-01


In [36]:
foo['Ticker'].nunique()

360

In [37]:
# checking market cap dist. of remaining companies
raw_companies.loc[raw_companies['ticker'].isin(foo['Ticker'].unique()), 'market_cap'].value_counts()

market_cap
Large-Cap    132
Mid-Cap      126
Small-Cap    102
Name: count, dtype: int64

In [ ]:
foo.to_csv("../data/raw/financial_data.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'data\raw'

# Text Data

Collection news info

In [ ]:
tickers = raw_companies['ticker'].tolist()

In [ ]:
raw_companies.sample()

In [ ]:
sample = yf.Ticker("XOM")

In [ ]:
len(sample.news)

In [ ]:
sample.news[0]['content']

In [ ]:
sample.news

In [ ]:
from datetime import datetime

In [ ]:
def get_stock_news(ticker_symbol: str, cutoff_date: str, max_articles: int = 100) -> pd.DataFrame:
    try:
        # Convert cutoff_date to datetime
        cutoff_dt = pd.to_datetime(cutoff_date)
        
        # Initialize ticker
        ticker = yf.Ticker(ticker_symbol)
        
        # Get news
        news = ticker.news
        
        if not news:
            return pd.DataFrame()
            
        # Extract relevant information from nested structure
        processed_news = []
        for article in news:
            content = article.get('content', {})
            provider = content.get('provider', {})
            thumbnail = content.get('thumbnail', {})
            
            processed_article = {
                'Title': content.get('title'),
                'Summary': content.get('summary'),
                'Date': pd.to_datetime(content.get('pubDate')),
                'Publisher': provider.get('displayName'),
                'Publisher_URL': provider.get('url'),
                'Content_Type': content.get('contentType'),
                'Is_Premium': content.get('finance', {}).get('premiumFinance', {}).get('isPremiumNews', False),
                'Article_URL': content.get('previewUrl'),
                'Thumbnail_URL': thumbnail.get('originalUrl') if thumbnail else None
            }
            processed_news.append(processed_article)
            
        # Convert to DataFrame
        news_df = pd.DataFrame(processed_news)
        
        # Filter articles by cutoff date
        # news_df = news_df[news_df['Date'] <= cutoff_dt]
        
        # Sort by date in descending order
        news_df = news_df.sort_values('Date', ascending=False)
        
        # Limit the number of articles
        news_df = news_df.head(max_articles)
        
        return news_df
        
    except Exception as e:
        print(f"Error fetching news for {ticker_symbol}: {str(e)}")
        return pd.DataFrame()

In [ ]:
news_df = get_stock_news("XOM", "2023-12-31")

In [ ]:
news_df

In [ ]:
len(sample.get_news(count=5))